In [ ]:
!pip install transformers
!pip install evaluate
!pip install datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import TrainingArguments
import evaluate
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import TrainingArguments


In [3]:
# Load labeled and unlabeled datasets
lab = pd.read_csv('/kaggle/input/fb-post/FB_posts_labeled.txt', sep='\t')
unlab = pd.read_csv('/kaggle/input/fb-post/FB_posts_unlabeled.txt', sep='\t')

# Preparing the labeled data
lab['label'] = 0
lab.loc[lab['Complaint'] == 1, 'label'] = 1
lab.loc[lab['Feedback'] == 1, 'label'] = 2
lab_dataset = lab[['message', 'label']]

# Create Dataset objects
train_ds = Dataset.from_pandas(lab_dataset, split="train")
test_ds = Dataset.from_pandas(unlab[['postId', 'message']], split="test")

# Tokenization and encoding the data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["message"], padding="max_length", truncation=True, max_length=512)

train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

# Define the model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/7961 [00:00<?, ? examples/s]

Map:   0%|          | 0/2039 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Training arguments
training_args = TrainingArguments(
    output_dir="model_output",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="logs"
)

# Define F1 score metric
f1_metric = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return f1_metric.compute(predictions=predictions, references=labels)


In [6]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    compute_metrics=compute_metrics,
    eval_dataset=test_ds
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,No log
2,0.096500,No log
3,0.051700,No log


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1494, training_loss=0.057786107861213735, metrics={'train_runtime': 1418.1912, 'train_samples_per_second': 16.84, 'train_steps_per_second': 1.053, 'total_flos': 6283937755597824.0, 'train_loss': 0.057786107861213735, 'epoch': 3.0})

In [ ]:
# Predict on the test dataset
predictions = trainer.predict(test_ds)
predicted_labels = np.argmax(predictions.predictions, axis=1)

In [ ]:
# Save predictions back to DataFrame
unlab['predicted_label'] = predicted_labels
unlab_final = pd.concat([unlab, pd.get_dummies(unlab['predicted_label'], prefix='category')], axis=1)
unlab_final.columns = ['postId', 'message', 'predicted_label', 'Appreciation_pred', 'Complaint_pred', 'Feedback_pred']
unlab_final = unlab_final[['postId', 'Appreciation_pred', 'Complaint_pred', 'Feedback_pred']]

# Save or print results
print(unlab_final.head())

In [ ]:
unlab_final.to_csv('uncasedHF.csv')
